# Captura de mudanças no ambiente através da webcam

In [ ]:
import cv2

# Abrir o vídeo
cam = cv2.VideoCapture(0)  # Troque pelo caminho do seu vídeo

# Ler o primeiro frame como referência de fundo
ret, frame_reference = cam.read()
if not ret:
    print("Erro ao carregar o vídeo")
    cam.release()
    exit()

frame_reference = cv2.flip(frame_reference, 1) 
# Converter o frame de referência para escala de cinza
frame_reference_gray = cv2.cvtColor(frame_reference, cv2.COLOR_BGR2GRAY)
frame_reference_gray = cv2.GaussianBlur(frame_reference_gray, (21, 21), 0)

while True:
    ret, frame = cam.read()

    if not ret:
        break
    
    frame = cv2.flip(frame, 1) 
    # frame = cv2.resize(frame, (0,0), fx=0.5, fy=0.5)
    # Converter o frame atual para escala de cinza e aplicar suavização
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.GaussianBlur(frame_gray, (21, 21), 0)

    # Subtrair o frame de referência do frame atual para detectar mudanças
    frame_delta = cv2.absdiff(frame_reference_gray, frame_gray)

    # Aplicar o thresholding para obter a imagem binária das mudanças
    _, thresh = cv2.threshold(frame_delta, 25, 255, cv2.THRESH_BINARY)

    # Dilatar a imagem para preencher buracos nos contornos
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Encontrar contornos das mudanças
    contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Desenhar retângulos ao redor das áreas detectadas
    for contour in contours:
        if cv2.contourArea(contour) < 500:  # Ignorar pequenas áreas
            continue
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Mostrar o frame com as detecções
    cv2.imshow('Deteccao de Mudancas', frame)
    cv2.imshow('Thresh', thresh)
    cv2.imshow('Frame Delta', frame_delta)

    # Pressione 'q' para sair
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Liberar o vídeo e fechar as janelas
cam.release()
cv2.destroyAllWindows()
